<a href="https://colab.research.google.com/github/LordLean/Tracking-Sources-Of-Online-Disinformation/blob/main/twitter_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import os
import tweepy as tw
import pandas as pd

In [2]:
consumer_key= 'JxciVDzwxuGOuCaBV9rw1fdg9'
consumer_secret= 'PBrBFjXxmw7cSim2OyaNIGVy51AjFnq50gF9VlCv0LQ4caCiAv'
access_token= '1317451540471447553-eFRWiyt3100ReKxW3mOrSDUtjy1iFu'
access_token_secret= 'DhGgoYwPzFYRygV3nZOj2GIlKcLIL7sWWhLBvUoz1LDDK'

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Start

In [4]:
def extract_coordinates(row):
  '''
  Extracts coordinates from tweet if not null.
  '''
  if row['Tweet Coordinates']:
      return row['Tweet Coordinates']['coordinates']
  else:
      return None


# Function created to extract place such as city, state or country from tweet if it has place info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_place(row):
  '''
  Extracts place from tweet if not null, in the form of state, city, country.
  '''
  if row['Place Info']:
      return row['Place Info'].full_name
  else:
      return None


def get_untruncated_tweet(id):
  '''
  Look up function which returns the untruncated tweet if not a retweet. 
  '''
  status = api.get_status(id, tweet_mode="extended")
  try:
      return status.retweeted_status.full_text
  except AttributeError:  # Not a Retweet
      return status.full_text

In [74]:
query = "Biden letter trump"

# Cursor arguments.
cursor_args = {
    "query" : query,
    "username" : "",
    "lang" : "en",
    "coordinates" : "40.7128, 74.0060, 100mi",
    "date_since" : "2020-11-16", # YYYY-MM-DD
    "max_tweets" : 1000
}

In [75]:
if cursor_args["username"]:
  # User Timeline
  tweets = tw.Cursor(api.user_timeline,
                    id = cursor_args["username"], 
                    tweet_mode = "extended"
                    ).items(cursor_args["max_tweets"])
else:
  # API Search - https://docs.tweepy.org/en/latest/api.html - ctrl+f "Search Methods"
  tweets = tw.Cursor(api.search,
                    q = cursor_args["query"],
                    # geocode = cursor_args["coordinates"],
                    result_type = "mixed", # mixed, recent, popular
                    lang = cursor_args["lang"],
                    include_entities = True,
                    tweet_mode = "extended"
                    ).items(cursor_args["max_tweets"])


# Pull info from tweets iterable.
final_list = []
for tweet in tweets:
  if hasattr(tweet, "retweeted_status") and tweet.entities["urls"]:
    tweet_info = [tweet.full_text, str(tweet.entities["urls"][0]), tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.user.location, tweet.lang, tweet.source, tweet.retweeted_status.created_at, tweet.retweeted_status.id_str, tweet.retweeted_status.user.screen_name, tweet.retweeted_status.user.location, tweet.retweeted_status.lang, tweet.retweeted_status.source]
    final_list.append(tweet_info)
  elif hasattr(tweet, "retweeted_status") and not tweet.entities["urls"]:
    tweet_info = [tweet.full_text, None, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.user.location, tweet.lang, tweet.source, tweet.retweeted_status.created_at, tweet.retweeted_status.id_str, tweet.retweeted_status.user.screen_name, tweet.retweeted_status.user.location, tweet.retweeted_status.lang, tweet.retweeted_status.source]
    final_list.append(tweet_info)
  elif not hasattr(tweet, "retweeted_status") and tweet.entities["urls"]:
    tweet_info = [tweet.full_text, str(tweet.entities["urls"][0]), tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.user.location, tweet.lang, tweet.source, None, None, None, None, None, None]
    final_list.append(tweet_info)


# Create dataframe for specified above information.
df = pd.DataFrame(final_list,columns=["RT Full Text", "Tweet Links", "RT Datetime", "RT Id", "RT User Handle", "RT User Location", "RT User Language", "RT User Source", "Orig Datetime", "Orig Tweet Id", "Orig User Handle", "Orig User Location", "Orig User Language", "Orig User Source"])

In [76]:
df

,RT Full Text,Tweet Links,RT Datetime,RT Id,RT User Handle,RT User Location,RT User Language,RT User Source,Orig Datetime,Orig Tweet Id,Orig User Handle,Orig User Location,Orig User Language,Orig User Source
0,Another GOP letter to Biden asking him to “fol...,"{'url': 'https://t.co/jc7N6f665z', 'expanded_u...",2021-01-29 21:00:53,1355259643338121218,igorbobic,"Washington, D.C.",en,Twitter for iPhone,NaT,None,None,None,None,None
1,RT @ARTHURGCARTER1: White House says Biden won...,None,2021-01-31 13:04:41,1355864579453177861,polytwonk,Englandshire,en,PolyTwonkBot,2021-01-31 12:58:43,1355863079834234883,ARTHURGCARTER1,INDIVIDUAL MERIT FIRST & LAST,en,Twitter Web App
2,White House says Biden won’t release ‘gracious...,"{'url': 'https://t.co/OeHmshw9X6', 'expanded_u...",2021-01-31 12:58:43,1355863079834234883,ARTHURGCARTER1,INDIVIDUAL MERIT FIRST & LAST,en,Twitter Web App,NaT,None,None,None,None,None
3,"RT @therecount: @kaitlancollins Biden: ""The pr...",None,2021-01-31 12:36:22,1355857454949666817,omarmuniz9,,en,Twitter for iPhone,2021-01-20 22:30:39,1352020742993563649,therecount,"New York, NY",en,Twitter Web App
4,"RT @onumazih: BIDEN: ""PRESIDENT TRUMP LEFT FOR...",None,2021-01-31 12:14:52,1355852043659915271,ngwumohike,,en,Twitter for Android,2021-01-31 10:44:06,1355829203590127621,onumazih,IN THE UNIVERSE,en,Twitter for Android
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Letter to editor: George McCaleb Wilkinson say...,"{'url': 'https://t.co/5hTfCIWv8f', 'expanded_u...",2021-01-29 14:30:07,1355161305196490756,TimKalich,"Greenwood, Mississippi",en,BLOX CMS,NaT,None,None,None,None,None
88,Letter to editor: George McCaleb Wilkinson say...,"{'url': 'https://t.co/QEje5NKPx5', 'expanded_u...",2021-01-29 14:30:07,1355161303875256321,gwcommonwealth,"Greenwood, Mississippi",en,BLOX CMS,NaT,None,None,None,None,None
89,RT @NewYorker: Donald Trump—and the thorough T...,None,2021-01-29 13:45:42,1355150127686508545,michaellamperd,"Minnesota, USA",en,Twitter Web App,2021-01-29 11:47:38,1355120414008074246,NewYorker,"New York, NY",en,SocialFlow
90,RT @RohdeD: Reflections on a post-Trump week o...,None,2021-01-29 13:45:30,1355150076620886032,rjsplb85,"West Summerlin, Las Vegas",en,Twitter for iPhone,2021-01-28 23:10:16,1354929817641226247,RohdeD,,en,Twitter Web App


In [80]:
df["Orig User Handle"].value_counts()

peterbakernyt      25
ABCPolitics        11
_Ikedinachi         4
NicholsUprising     3
NewYorker           2
DailyMail           2
candies2639         2
ThisWeekABC         2
fidh_en             2
RepMikeQuigley      1
RepMaloney          1
RepBarragan         1
cmsub               1
greenfield64        1
MostafaMe4          1
CourierNewsroom     1
RohdeD              1
collinlaverty11     1
ARTHURGCARTER1      1
IndyUSA             1
DJJudd              1
patkiernan          1
onumazih            1
therecount          1
Name: Orig User Handle, dtype: int64

In [9]:
print(df.at[0,"RT Full Text"])
id = df.at[0, "RT Id"]
print(get_untruncated_tweet(id))

RT @goyalsanjeev: The #FakeNews industry is unstoppable ... One can't even imagine where this industry attack and when😎

Instead of focusin…
The #FakeNews industry is unstoppable ... One can't even imagine where this industry attack and when😎

Instead of focusing their time fully on investigations, agencies have to spend time exposing fake-news also.

Reason&gt;ZERO FEAR of @narendramodi!
@PrakashJavdekar @dir_ed https://t.co/dNMklMpfne


# Redundant

In [ ]:
# # Collect tweets
# tweets = tw.Cursor(api.search,
#                        q=search_words,
#                        lang="en"
#                        ).items(10000)

# # Collect a list of tweets
# all_tweets = [tweet.user for tweet in tweets]

In [ ]:
# from geopy.geocoders import Nominatim
# import gmplot

# geolocator = Nominatim()

# coordinates = {'latitude': [], 'longitude': []}
# for count, user_loc in enumerate(tweets.location):
#     try:
#         location = geolocator.geocode(user_loc)
        
#         # If coordinates are found for location
#         if location:
#             coordinates['latitude'].append(location.latitude)
#             coordinates['longitude'].append(location.longitude)
            
#     # If too many connection requests
#     except:
#         pass

In [ ]:
# # Old
# if cursor_args["username"]:
#   # User Timeline
#   tweets = tw.Cursor(api.user_timeline,
#                     id = cursor_args["username"], 
#                     tweet_mode = 'extended'
#                     ).items(cursor_args["max_tweets"])
# else:
#   # API Search - https://docs.tweepy.org/en/latest/api.html - ctrl+f "Search Methods"
#   tweets = tw.Cursor(api.search,
#                     q = cursor_args["query"],
#                     # geocode = cursor_args["coordinates"],
#                     result_type = "mixed", # mixed, recent, popular
#                     lang = cursor_args["lang"],
#                     tweet_mode = 'extended'
#                     ).items(cursor_args["max_tweets"])

# # tweets2 = deepcopy(tweets)
# # for tweet in tweets2:
# #   # print(tweet.retweeted_status)
# #   pass

# # Pulling information from tweets iterable object
# # Add or remove tweet information you want in the below list comprehension
# tweets_list = [[tweet.full_text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.retweeted_status.user.Name, tweet.user.location, tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang, tweet.source, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id_str, tweet.is_quote_status] for tweet in tweets if hasattr(tweet,"retweeted_status")]
 
# # Creation of dataframe from tweets_list
# # Add or remove columns as you remove tweet information
# tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', "Retweeted Tweet", "Location", 'Tweet Coordinates', 'Place Info', 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id', 'Replied Tweet User Id Str', 'Quote Status Bool'])
 
# # Checks if there are coordinates attached to tweets, if so extracts them
# tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
 
# # Checks if there is place information available, if so extracts them
# tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)

In [ ]:
# if cursor_args["username"]:
#   # User Timeline
#   tweets = tw.Cursor(api.user_timeline,
#                     id = cursor_args["username"], 
#                     tweet_mode = "extended"
#                     ).items(cursor_args["max_tweets"])
# else:
#   # API Search - https://docs.tweepy.org/en/latest/api.html - ctrl+f "Search Methods"
#   tweets = tw.Cursor(api.search,
#                     q = cursor_args["query"],
#                     # geocode = cursor_args["coordinates"],
#                     result_type = "mixed", # mixed, recent, popular
#                     lang = cursor_args["lang"],
#                     include_entities = True,
#                     tweet_mode = "extended"
#                     ).items(cursor_args["max_tweets"])


# # Pull info from tweets iterable.
# tweets_list = [[tweet.full_text, tweet.entities["urls"][0], tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.user.location, tweet.lang, tweet.source, tweet.retweeted_status.user.screen_name, tweet.retweeted_status.created_at, tweet.retweeted_status.id_str, tweet.retweeted_status.user.screen_name, tweet.retweeted_status.user.location, tweet.retweeted_status.lang, tweet.retweeted_status.source] for tweet in tweets if hasattr(tweet,"retweeted_status") and tweet.entities["urls"]]
 
# # Create dataframe for specified above information.
# df = pd.DataFrame(tweets_list,columns=["RT Full Text", "Tweet Links", "RT Datetime", "RT Id", "RT User Handle", "RT User Location", "RT User Language", "RT User Source", "Orig User Handle", "Orig Datetime", "Orig Tweet Id", "Orig User Handle", "Orig User Location", "Orig User Language", "Orig User Source"])

In [10]:
# from google.colab import files
# import gmplot
# from geopy.geocoders import Nominatim

# locations = list(df["RT User Location"].unique())

# geolocator = Nominatim(user_agent="http")

# # Go through all tweets and add locations to 'coordinates' dictionary
# coordinates = {'latitude': [], 'longitude': []}
# for count, user_loc in enumerate(locations):
#     try:
#         location = geolocator.geocode(user_loc)
        
#         # If coordinates are found for location
#         if location:
#             coordinates['latitude'].append(location.latitude)
#             coordinates['longitude'].append(location.longitude)
            
#     # If too many connection requests
#     except:
#         pass
    
# # Instantiate and center a GoogleMapPlotter object to show our map
# gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# # Insert points on the map passing a list of latitudes and longitudes
# gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# # Save the map to html file
# gmap.draw("python_heatmap.html")

# # Download html
# files.download("python_heatmap.html")